[Official Tutorial](https://www.tensorflow.org/get_started/get_started)
<br>
[Batch size](https://www.zhihu.com/question/32673260)

In [4]:
# https://www.tensorflow.org/get_started/get_started
import tensorflow as tf
from tensorflow import estimator

### tf.constant

In [10]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [11]:
node1??

In [4]:
# A `Session` object encapsulates the environment in which `Operation` objects are executed, and `Tensor` objects are evaluated.
sess = tf.Session()

In [17]:
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456)]

In [14]:
a = sess.run((node1, node2))
b = sess.run([node1, node2])
[a, b, type(a), type(b)]

[(3.0, 4.0), [3.0, 4.0], tuple, list]

In [18]:
# combining Tensor nodes with operations(Operations are also nodes)
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


### tf.placeholder
* A placeholder is a promise to provide a value *later*

In [19]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + provides a shortcut for tf.add(a,b)

In [20]:
# evaluate this graph with multiple inputs by using the feed_dict argument to the run method
print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1,3], b:[2,4]}))

7.5
[ 3.  7.]


In [21]:
# adding another operation
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:3, b:4.5}))

22.5


### tf.Variables
* Variables allow us to add trainable parameters to a graph. They are contructed with a type and initial value

In [22]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

* Constants are initialized when you call tf.constant, and their value can never change.
* By contrast, variables are not initialized when you call tf.Variable.
* To initialize all the variables in a TensorFlow program, you must explicity call a special operation as follows:

In [23]:
init = tf.global_variables_initializer()
sess.run(init)  # until we call sess.run, the variables are uninitialized

In [24]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [26]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))
print(sess.run(squared_deltas, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66
[  0.           1.68999982   6.75999928  15.21000099]


* tf.Variable can be changed using operations like **tf.assign**

In [23]:
fixW = tf.assign(W,[-1.])
fixb = tf.assign(b,[1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1,2,3,4], y: [0,-1,-2,-3]}))

0.0


### tf.train API :optimizers
* Tensorflow provides optimizers that slowly change each variable in order to minimize the loss function
* the simplest optimizer is gradient descent

In [24]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1,2,3,4], y: [0,-1,-2,-3]})
print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [26]:
# evaluate training accuracy
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
curr_W, curr_b, curr_loss = sess.run([W,b,loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s" %(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


### tf.estimator  *tf.estimator defines many common models*
tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, 
* running training loops
* running evaluation loops
* managing data sets

In [28]:
import numpy as np
import tensorflow as tf

In [30]:
# declare list of features
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

In [31]:
# The following code provides an estimator that does linear regression
estimator = tf.estimator.LinearRegressor(feature_columns = feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\luan\\AppData\\Local\\Temp\\tmpamaw6pjr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021C999285F8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:
# Here we use two data sets: one for training and one for evaluation
x_train = np.array([1.,2.,3.,4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2.,5.,8.,1.])
y_eval = np.array([-1.01,-4.1,-7,0.])

input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\luan\AppData\Local\Temp\tmpamaw6pjr\model.ckpt.
INFO:tensorflow:loss = 21.0, step = 1
INFO:tensorflow:global_step/sec: 682.588
INFO:tensorflow:loss = 0.240081, step = 101 (0.162 sec)
INFO:tensorflow:global_step/sec: 709.221
INFO:tensorflow:loss = 0.07524, step = 201 (0.129 sec)
INFO:tensorflow:global_step/sec: 790.509
INFO:tensorflow:loss = 0.0148608, step = 301 (0.125 sec)
INFO:tensorflow:global_step/sec: 813.01
INFO:tensorflow:loss = 0.00316879, step = 401 (0.124 sec)
INFO:tensorflow:global_step/sec: 651.46
INFO:tensorflow:loss = 0.00155151, step = 501 (0.155 sec)
INFO:tensorflow:global_step/sec: 826.459
INFO:tensorflow:loss = 0.000113987, step = 601 (0.120 sec)
INFO:tensorflow:global_step/sec: 719.395
INFO:tensorflow:loss = 9.54682e-05, step = 701 (0.138 sec)
INFO:tensorflow:global_step/sec: 544.97
INFO:tensorflow:loss = 4.0516e-05, step = 801 (0.187 sec)
INFO:tensorflow:global_step/se